In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
df=pd.read_csv('./Dataset/zomato.csv/zomato.csv', encoding='latin-1')

print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51717 entries, 0 to 51716
Data columns (total 17 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   url                          51717 non-null  object
 1   address                      51717 non-null  object
 2   name                         51717 non-null  object
 3   online_order                 51717 non-null  object
 4   book_table                   51717 non-null  object
 5   rate                         43942 non-null  object
 6   votes                        51717 non-null  int64 
 7   phone                        50509 non-null  object
 8   location                     51696 non-null  object
 9   rest_type                    51490 non-null  object
 10  dish_liked                   23639 non-null  object
 11  cuisines                     51672 non-null  object
 12  approx_cost(for two people)  51371 non-null  object
 13  reviews_list                 51

In [3]:
import os
print(os.getcwd())

c:\Users\GC\Desktop\Data Science Become Data Scientist full\Data Analysis self projects\Zomate Hotel Analysis


In [4]:
# Clean 'rate' column
df = df[df['rate'].notnull()]
df = df[~df['rate'].isin(['NEW', '-'])]
df['rate'] = df['rate'].apply(lambda x: float(x.split('/')[0]))

# Drop rows with missing cuisines or location
df = df[df['cuisines'].notnull()]
df = df[df['location'].notnull()]


In [5]:
df.to_csv('./Dataset/updated_zomato.csv', index=False)

In [6]:
#Task 1: Cuisine vs Rating (Plotly)

# Prepare data
df_cuisine = df.copy()
df_cuisine['cuisines'] = df_cuisine['cuisines'].apply(lambda x: [i.strip() for i in x.split(',')])
df_cuisine = df_cuisine.explode('cuisines')

# Group by cuisine and calculate mean rating
cuisine_rating = df_cuisine.groupby('cuisines')['rate'].mean().sort_values(ascending=False).reset_index()

# Top 15 cuisines
top_cuisines = cuisine_rating.head(15)

# Plot using Plotly
fig1 = px.bar(top_cuisines, 
              x='rate', 
              y='cuisines', 
              orientation='h',
              color='rate',
              color_continuous_scale='Viridis',
              title='Top 15 Cuisines by Average Rating')
fig1.update_layout(yaxis={'categoryorder':'total ascending'})
fig1.show()


In [7]:
# Task 2: Location-wise Best Restaurants (Plotly)

# Filter high-rated and popular restaurants
top_rated = df[(df['rate'] >= 4.5) & (df['votes'] >= 100)]

# Count top restaurants per location
top_by_location = top_rated.groupby('location')['name'].count().sort_values(ascending=False).head(10).reset_index()

# Plot
fig2 = px.bar(top_by_location, 
              x='location', 
              y='name', 
              color='name',
              color_continuous_scale='sunset',
              title='Top 10 Locations with Most Highly Rated Restaurants')
fig2.update_layout(xaxis_title='Location', yaxis_title='Number of Top Restaurants')
fig2.show()


HTML plots for hovering 

In [8]:
# all plots in single html file

import pandas as pd
import plotly.express as px
from wordcloud import WordCloud
from PIL import Image
import io
import plotly.io as pio
from plotly.subplots import make_subplots
from plotly.graph_objs import Figure
import os


In [9]:

# --- Plot 1: Top 15 Cuisines by Average Rating ---
fig1 = px.bar(top_cuisines, 
              x='rate', 
              y='cuisines', 
              orientation='h',
              color='rate',
              color_continuous_scale='Viridis',
              title='Top 15 Cuisines by Average Rating')
fig1.update_layout(yaxis={'categoryorder':'total ascending'})

# --- Plot 2: Top 10 Locations with Most Highly Rated Restaurants ---
# Plot
fig2 = px.bar(top_by_location, 
              x='location', 
              y='name', 
              color='name',
              color_continuous_scale='sunset',
              title='Top 10 Locations with Most Highly Rated Restaurants')
fig2.update_layout(xaxis_title='Location', yaxis_title='Number of Top Restaurants')


# Save all plots in a single HTML file
#html_path = "Dataset/netflix_analysis_all_plots.html"

#pio.write_html([fig1, fig2, fig3, fig4, fig5, fig6, fig7, fig8, fig9], file=html_path, auto_open=False)

#html_path


In [10]:
import plotly.io as pio

# List of all your figures
figures = [fig1,fig2]

# Generate HTML strings for each figure
html_parts = [pio.to_html(fig, full_html=False, include_plotlyjs='cdn') for fig in figures]

# Combine them into one full HTML document
full_html = f"""
<html>
<head>
    <title>Hotel Analysis</title>
</head>
<body>
    {''.join(html_parts)}
</body>
</html>
"""

# Save to file
output_path = "Hotel_analysis_all_plots.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"Saved to {output_path}")


Saved to Hotel_analysis_all_plots.html
